# OR3 PZ: Access Object Catalogs

Author: Melissa Graham

Last verified to run: Fri Apr 12 2024

LSST Science Pipelines version: Weekly 2024_04

**Overview** 

The contents of this notebook have relied on the
<a href="https://github.com/lsst-sitcom/ops_rehearsal_commissioning_2024/blob/main/notebooks/ops_rehearsal_comcam_analysis.ipynb">ops_rehearsal_comcam_analysis notebook</a>.

This notebook shows how to access OR3 Object catalogs via the butler. 

It also explores the tract coverage and depths and the types of flux measurements available.

## Set up

Import packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
import gc

Define the Rubin filter names, colors, and symbols to use in plots.

In [ ]:
flbl = ['u', 'g', 'r', 'i', 'z', 'y']
fclr = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
        'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
fsym = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

## Access OR3 DRP data

### Find the object catalog

Based on the information in this Confluence page: https://confluence.lsstcorp.org/display/DM/Campaigns,
the Data Release Processing (DRP) for simulated ComCam data at USDF was complete back in March.

It is DRP that creates the deepCoadds and Object catalog which is the starting point for photo-z estimates.

Based on this Slack thread, this early processing was done in order to create template images for OR3,
but it is unclear if "final" DRP-type processing will be done for OR3.
https://lsstc.slack.com/archives/C011E3ZUZAB/p1712762539582939

In [ ]:
repo = '/repo/ops-rehearsal-3-prep'
collection = 'u/homer/w_2024_12/DM-43439'
butler = Butler(repo, collections=collection)
registry = butler.registry

Determine which `DatasetTypes` exist in the collection.

Limit the search to the data products, and do not list configurations, logs, etc.

In [ ]:
# for datasetType in registry.queryDatasetTypes():
#     if registry.queryDatasets(datasetType, 
#                               collections=collection).any(execute=False,
#                                                           exact=False):
#         if ('_config' not in datasetType.name) and \
#         ('_log' not in datasetType.name) and \
#         ('_metadata' not in datasetType.name) and \
#         ('_resource_usage' not in datasetType.name):
#             print(datasetType)

Only look for the `object`-related data products.

The following cell shows that there is `objectTable` and `objectTable_tract`, plus a whole bunch of related datasets.

In [ ]:
# for datasetType in registry.queryDatasetTypes():
#     if registry.queryDatasets(datasetType, 
#                               collections=collection).any(execute=False,
#                                                           exact=False):
#         if ('_config' not in datasetType.name) and \
#         ('_log' not in datasetType.name) and \
#         ('_metadata' not in datasetType.name) and \
#         ('_resource_usage' not in datasetType.name):
#             temp = str(datasetType.name)
#             if temp.find('object') > -1:
#                 print(temp)

Alternatively, can do it this way and reach the same conclusion.

In [ ]:
# for dtype in sorted(registry.queryDatasetTypes(expression="*object*")):
#     print(dtype.name)

Get all the butler references for the `objectTable_tract`.

In [ ]:
oTt_refs = list(butler.registry.queryDatasets('objectTable_tract'))

What are the `dataId` composed of, for the object table?

They would be all the same, so just check the first.

In [ ]:
for i, ref in enumerate(oTt_refs):
    if i == 0:
        print(ref.dataId)

## Characterize object catalog

### Number of tracts and visits/tract

How many unique tracts are covered by `objectTable_tract`.

In [ ]:
tracts = np.unique([ref.dataId['tract'] for ref in oTt_refs])
print('Number of unique tracts: ', len(tracts))
print('tracts: ', tracts)

How many visits were available for the `deepCoadd` images in each tract.

See that the numbers go from <10 to >1000.
Some tracts will not have enough visits to even coadd (yet, an `objectTable` was made for them...).
This shows depth variation over the full region is to be expected.

In [ ]:
temp = []
for tract in tracts:
    visits = list(butler.registry.queryDatasets('visitSummary', tract=tract, 
                                                skymap='DC2', findFirst=True))
    temp.append(len(visits))
nvisits = np.asarray(temp, dtype='int')
del temp
sx = np.argsort(nvisits)
for x in sx:
    print(tracts[x], nvisits[x])

Plot histogram of the number of tracts (y) vs. number of visits/tract (x).

In [ ]:
fig = plt.figure(figsize=(3, 2))
plt.hist(nvisits, bins=20)
plt.xlabel('number of visits per tract')
plt.ylabel('number of tracts')
plt.show()

### Retrieve object catalog contents

How to access the `objectTable_tract` for a given tract.

In [ ]:
use_tract = 9880
dataId = {'skymap': 'ops_rehersal_prep_2k_v1', 'tract': use_tract}
objects = butler.get('objectTable_tract', dataId=dataId)

Option to show (truncated) table.

Schema is going to be very similar to the DP0.2 Object table.

https://dm.lsst.org/sdm_schemas/browser/dp02.html#Object

In [ ]:
# objects

Number of columns, number of rows.

In [ ]:
print('# cols: ', len(objects.columns))
print('# rows: ', len(objects))

Search column names.

In [ ]:
string = 'ext'
for col in objects.columns:
    if col.find(string) > 0:
        print(col)

### Plot r- and i-band cModel magnitude distributions

Extract data into numpy arrays for analysis.

The best way to reject non-unique objects is to impose that `detect_isPrimary` be `True`.
This removes duplicates at image edge overlaps and deblending parents.

In [ ]:
r_cModelFlux = np.asarray(objects.get('r_cModelFlux'))
i_cModelFlux = np.asarray(objects.get('i_cModelFlux'))
detect_isPrimary = np.asarray(objects.get('detect_isPrimary'))

Calculate magnitudes for the subset of r- and i-band detected unique `objects`.

In [ ]:
tx = np.where((r_cModelFlux > 0.0) & (i_cModelFlux > 0.0) & (detect_isPrimary == 1))[0]
print(len(tx))
r_cModelMag = -2.50 * np.log10(r_cModelFlux[tx]) + 31.4
i_cModelMag = -2.50 * np.log10(i_cModelFlux[tx]) + 31.4
del tx

Plot the magnitude distribution.

In [ ]:
tx = np.where((r_cModelMag < 50) & (i_cModelMag < 50))[0]
fig = plt.figure(figsize=(3, 2))
plt.hist(r_cModelMag[tx], bins=20, log=True, histtype='step', lw=2, alpha=0.5, color=fclr['r'], label='r')
plt.hist(i_cModelMag[tx], bins=20, log=True, histtype='step', lw=1, color=fclr['i'], label='i')
plt.xlabel('mag')
plt.ylabel('N')
plt.legend(loc='upper right')
plt.title('tract = '+str(use_tract))
plt.show()
del tx

Clean up.

In [ ]:
del dataId, objects
del r_cModelFlux, i_cModelFlux, detect_isPrimary, r_cModelMag, i_cModelMag
gc.collect()

Plot distribution of r-band magnitudes for the following tracts (# visits):

```
9881 8
7684 60
9638 299
7149 602
9880 1280
```

In [ ]:
use_tracts = [9881, 7684, 9638, 7149, 9880]
use_nvisits = [8, 60, 299, 602, 1280]

fig = plt.figure(figsize=(6, 4))

for i, tract in enumerate(use_tracts):
    dataId = {'skymap': 'ops_rehersal_prep_2k_v1', 'tract': tract}
    objects = butler.get('objectTable_tract', dataId=dataId)
    
    r_cModelFlux = np.asarray(objects.get('r_cModelFlux'))
    detect_isPrimary = np.asarray(objects.get('detect_isPrimary'))
    tx = np.where((r_cModelFlux > 0.0) & (detect_isPrimary == 1))[0]
    r_cModelMag = -2.50 * np.log10(r_cModelFlux[tx]) + 31.4
    del tx
    
    tx = np.where(r_cModelMag < 50)[0]
    plt.hist(r_cModelMag[tx], bins=20, log=True, histtype='step',
             label=str(use_nvisits[i]))
    
    del dataId, objects
    del r_cModelFlux, detect_isPrimary, r_cModelMag
    gc.collect()

plt.xlabel('r mag')
plt.ylabel('N')
plt.title('magnitude distribution by tract')
plt.legend(loc='upper left', title='# visits')
plt.show()

Going forward, take an ~arbitrary cut off of 500 visits contributing to a tract.

## Schema: types of fluxes measured

Schema for the object catalog for DP0.2: https://dm.lsst.org/sdm_schemas/browser/dp02.html#Object

Could be a bit different for OR3 but generally the same types of fluxes will have been measured.

For photometric redshifts, since accurate colors are important, it is the GaaP fluxes that should be used.


### Aperture fluxes

Fixed aperture diameter size in pixels.

```
<f>_ap<pix>Flux     : Flux within <pix>-pixel aperture. Forced on <f>-band.
<f>_ap<pix>FluxErr  : Uncertainty of <f>_ap<pix>Flux.
<f>_ap<pix>FluxFlag : Failure flag for <f>_ap<pix>Flux.
```

For DP0.2, the apertures are 3, 6, 9, 12, 17, 25, 35, 50, and 70 pixels.

In the column name, apertures are `03`, `06`, `09`, `12`, and so on.

### Composite Model (CModel) fluxes

Similar in nature to those measured for SDSS: 
https://www.sdss3.org/dr8/algorithms/magnitudes.php#cmodel

In short, it is the linear combination of the best fit exponential and de Vaucouleurs profiles.

```
<f>_cModelFlux    :	Flux from the final cmodel fit. Forced on <f>-band.
<f>_cModelFluxErr : Uncertainty of <f>_cModelFlux
<f>_cModel_flag   : Failure flag for <f>_cModelFlux
```

Fluxes fit to the individual model components.

```
<f>_bdFluxB    : Flux from the de Vaucouleurs fit. Measured on <f>-band.
<f>_bdFluxD    : Flux from the exponential fit. Measured on <f>-band.
<f>_bdFluxBErr : Uncertainty of <f>_bdFluxB
<f>_bdFluxDErr : Uncertainty of <f>_bdFluxD
```

The fit sizes are also available (half-light radii, ellipse axes).

### GaaP fluxes

The Gaussian-aperture-and-PSF flux from <a href="https://ui.adsabs.harvard.edu/abs/2008A%26A...482.1053K/abstract">Kuijken et al. 2008</a>.

**Optimal**

```
<f>_gaapOptimalFlux    : GaaP flux with optimal aperture after multiplying the seeing by 1.15. Forced on <f>-band.
<f>_gaapOptimalFluxErr : Uncertainty of <f>_gaapOptimalFlux.
```

**PSF**

```
<f>_gaapPsfFlux    : GaaP flux with PSF aperture after multiplying the seeing by 1.15. Forced on <f>-band.
<f>_gaapPsfFluxErr : Uncertainty of <f>_gaapPsfFlux.
```

**Aperture**

```
<f>_gaap<ap>Flux    : GaaP flux with <ap> aperture after multiplying the seeing by 1.15. Forced on <f>-band.
<f>_gaap<ap>FluxErr : Uncertainty of <f>_gaap<ap>Flux.
```

Where the apertures are 0.5, 0.7, 1.0, 1.5, 2.5, and 3.0.
In the column name `<ap>` appears as `0p5`, `0p7`, etc.




### Kron fluxes

A decent summary of Kron fluxes <a href="https://ned.ipac.caltech.edu/level5/March05/Graham/Graham2_6.html">in the NED documentation</a>.

```
<f>_kronFlux      : Flux from Kron Flux algorithm. Measured on <f> g-band.
<f>_kronFluxErr   : Uncertainty of <f>_kronFlux.
<f>_kronFlux_flag : Failure flag for <f>_kronFlux.
```

The Kron radius, `<f>_kronRad`, is also available.

### PSF fluxes

Fluxes derived using the model point-spread function (PSF) of the image.

```
<f>_psfFlux      : Flux derived from linear least-squares fit of PSF model. Forced on <f>-band.
<f>_psfFluxErr   : Uncertainty of <f>_psfFlux.
<f>_psfFlux_flag : Failure flag for <f>_psfFlux.

```


### Calibration fluxes

I believe these are the fluxes used for calibrations.

```
<f>_calibFlux      : Flux within 12.0-pixel aperture. Measured on <f>-band.
<f>_calibFluxErr   : Uncertainty of <f>_calibFlux.
<f>_calibFlux_flag : Failure flag for <f>_calibFlux.
```

### Blendedness and extendedness parameters

For blendedness, see also section 4.9.11 of <a href="https://ui.adsabs.harvard.edu/abs/2018PASJ...70S...5B/abstract">Bosch et al. 2018</a>.
Extendedness parameters are known to not be perfect (tested on DP0.2 in <a href="https://github.com/rubin-dp0/delegate-contributions-dp02/tree/main/extendedness">this notebook</a>).

```
<f>_blendedness  : Measure of how much the flux is affected by neighbors, (1 - child_flux/parent_flux). Measured on <f>-band.
<f>_extendedness : Set to 1 for extended sources, 0 for point sources. Measured on <f>-band.
```


## Explore the flux measurements

### Which flux type columns are populated?

Based on the cells below, it looks like all flux columns are populated.

Not demonstrated, but columns for filters u, z, and y are not populated.

In [ ]:
use_tract = 9880
dataId = {'skymap': 'ops_rehersal_prep_2k_v1', 'tract': use_tract}
objects = butler.get('objectTable_tract', dataId=dataId)

In [ ]:
i_ap12_flux = np.asarray(objects.get('i_ap12Flux'))
i_cmod_flux = np.asarray(objects.get('i_cModelFlux'))
i_gaap_flux = np.asarray(objects.get('i_gaapOptimalFlux'))
i_kron_flux = np.asarray(objects.get('i_kronFlux'))
i_psf_flux  = np.asarray(objects.get('i_psfFlux'))

In [ ]:
fig = plt.figure(figsize=(8, 4))
plt.hist(i_ap12_flux, bins=20, log=True, histtype='step', color=fclr['u'], label='ap12')
plt.hist(i_cmod_flux, bins=20, log=True, histtype='step', color=fclr['g'], label='cmod')
plt.hist(i_gaap_flux, bins=40, log=True, histtype='step', color=fclr['r'], label='gaap')
plt.hist(i_kron_flux, bins=20, log=True, histtype='step', color=fclr['i'], label='kron')
plt.hist(i_psf_flux, bins=20, log=True, histtype='step', color=fclr['z'], label='psf')
plt.xlabel('flux')
plt.ylabel('# objects')
plt.legend(loc='upper right', title='type')
plt.show()

In [ ]:
del i_ap12_flux, i_cmod_flux, i_gaap_flux, i_kron_flux, i_psf_flux

### Plot GaaP optimal VS GaaP 1.0 aperture

In [ ]:
i_gaap_optflux = np.asarray(objects.get('i_gaapOptimalFlux'))
i_gaap_1p0flux = np.asarray(objects.get('i_gaap1p0Flux'))

tx = np.where((i_gaap_optflux > 0.0) & (i_gaap_1p0flux > 0.0))[0]
i_gaap_optmag = -2.5 * np.log10(i_gaap_optflux[tx]) + 31.4
i_gaap_1p0mag = -2.5 * np.log10(i_gaap_1p0flux[tx]) + 31.4
del tx

In [ ]:
fig = plt.figure(figsize=(4, 4))
plt.plot(i_gaap_optmag, i_gaap_optmag-i_gaap_1p0mag, 'o', ms=2, mew=0, alpha=0.1, color='black')
plt.xlabel('opt')
plt.ylabel('opt - ap')
plt.show()

# fig = plt.figure(figsize=(4, 4))
# plt.hist2d(i_gaap_optmag, i_gaap_optmag-i_gaap_1p0mag, bins=30, density=True, norm='log', cmap='Greys')
# plt.colorbar()
# plt.xlabel('opt')
# plt.ylabel('opt - ap')
# plt.show()

Zoom in.

In [ ]:
fig = plt.figure(figsize=(4, 4))
plt.axhline(0.1, color='magenta')
plt.axhline(-0.1, color='magenta')
plt.plot(i_gaap_optmag, i_gaap_optmag-i_gaap_1p0mag, 'o', ms=2, mew=0, alpha=0.1, color='black')
plt.xlim([15, 28])
plt.ylim([-0.5, 0.5])
plt.xlabel('opt')
plt.ylabel('opt - ap')
plt.show()

Does the difference in optimal and aperture 1.0 GaaP fluxes depend on object extendedness? 

These plots show: yes it does.

In [ ]:
extendedness = np.asarray(objects.get('i_extendedness'))
tx = np.where((i_gaap_optflux > 0.0) & (i_gaap_1p0flux > 0.0))[0]
temp = extendedness[tx]

fig = plt.figure(figsize=(4, 4))
tx1 = np.where(temp <= 0.5)[0]
plt.plot(i_gaap_optmag[tx1], i_gaap_optmag[tx1]-i_gaap_1p0mag[tx1], 'o', ms=2, mew=0, alpha=0.1, color='blue')
plt.xlim([15, 28])
plt.ylim([-4, 1])
plt.xlabel('opt')
plt.ylabel('opt - ap')
plt.title('point-like objects')
plt.show()
del tx1

fig = plt.figure(figsize=(4, 4))
tx2 = np.where(temp > 0.5)[0]
plt.plot(i_gaap_optmag[tx2], i_gaap_optmag[tx2]-i_gaap_1p0mag[tx2], 'o', ms=2, mew=0, alpha=0.1, color='darkorange')
plt.xlim([15, 28])
plt.ylim([-4, 1])
plt.xlabel('opt')
plt.ylabel('opt - ap')
plt.title('extended objects')
plt.show()
del tx2

del tx, temp

What is the fraction of objects "affected by" non-matching optimal and aperture GaaP magnitudes?

Here we use a difference of 0.1 mag between optimal and aperture GaaP magnitudes to mean "affected by".

It's 37%, which is large. Unclear what "optimal" means

In [ ]:
diffs = i_gaap_optmag-i_gaap_1p0mag
tx = np.where(np.absolute(diffs) < 0.1)[0]
print(len(tx), len(i_gaap_optmag), len(tx)/len(i_gaap_optmag))
del tx

In [ ]:
del i_gaap_optflux, i_gaap_1p0flux
del i_gaap_optmag, i_gaap_1p0mag

### Multi-band GaaP magnitude distribution

In [ ]:
detect_isPrimary = np.asarray(objects.get('detect_isPrimary'))
extendedness = np.asarray(objects.get('i_extendedness'))
g_flux = np.asarray(objects.get('g_gaapOptimalFlux'))
r_flux = np.asarray(objects.get('r_gaapOptimalFlux'))
i_flux = np.asarray(objects.get('i_gaapOptimalFlux'))

tx = np.where((detect_isPrimary == 1) & (extendedness > 0) & 
              (g_flux > 0.0) & (r_flux > 0.0) & (i_flux > 0.0))[0]
print('Number of extended unique objects detected in gri: ', len(tx))

g_mag = -2.50 * np.log10(g_flux[tx]) + 31.4
r_mag = -2.50 * np.log10(r_flux[tx]) + 31.4
i_mag = -2.50 * np.log10(i_flux[tx]) + 31.4

In [ ]:
fig = plt.figure(figsize=(6, 4))
plt.hist(g_mag, bins=20, log=True, histtype='step', color=fclr['g'], label='g')
plt.hist(r_mag, bins=20, log=True, histtype='step', color=fclr['r'], label='r')
plt.hist(i_mag, bins=20, log=True, histtype='step', color=fclr['i'], label='i')
plt.xlabel('GaaP magnitude')
plt.ylabel('# extended sources')
plt.legend(loc='upper left', title='filter')
plt.show()

Clean up.

In [ ]:
del g_flux, r_flux, i_flux
del detect_isPrimary, extendedness
del g_mag, r_mag, i_mag
del use_tract, dataId, objects
gc.collect()